# MATH2504 S2, 2024, Project 1 Submission

### Trisztan Harai, 47483073

<html>
    <a href="https://github.com/TooMuchWater78/Trisztan-Harai-2504-2024-PROJECT1.git">
    GitHub repo
    </a>
<html>

## Task 1

## Task 2

When changing `max_degree_allowed` from `400` to `100`, and error occurs because during the

## Task 3

## Task 4

## Task 5

## Task 6

## Task 7

In [1]:
using Random

Random.seed!(0)

rand(4)

4-element Vector{Float64}:
 0.4056994708920292
 0.06854582438651502
 0.8621408571954849
 0.08597086585842195